In [1]:
!pip install tensorflow scikit-learn pandas numpy pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=256391 sha256=5a149d9168884c4b865a4a9ce05fb4610b7a0ce8978fc411088d5b975c6cc2c9
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
import pickle5 as pickle

In [3]:
df = pd.read_csv('/content/tripadvisor_hotel_reviews.csv')
print(df.head())

                                              Review  Rating
0  nice hotel expensive parking got good deal sta...       4
1  ok nothing special charge diamond member hilto...       2
2  nice rooms not 4* experience hotel monaco seat...       3
3  unique, great stay, wonderful time hotel monac...       5
4  great stay great stay, went seahawk game aweso...       5


In [4]:
df = df[['Review', 'Rating']]
df['sentiment'] = df['Rating'].apply(lambda x: 'positive' if x > 3
                                    else 'negative' if x < 3
                                    else 'neutral')
df = df[['Review', 'sentiment']]
df = df.sample(frac=1).reset_index(drop=True)

In [5]:
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['Review'])
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(df['Review'])
padded_sequences = pad_sequences(sequences, maxlen=100, truncating='post')

In [6]:
# one shot encoding
sentiment_labels = pd.get_dummies(df['sentiment']).values

In [7]:
x_train, x_test, y_train, y_test = train_test_split(padded_sequences, sentiment_labels, test_size=0.2)

In [8]:
model = Sequential()
model.add(Embedding(5000, 100, input_length=100))
model.add(Conv1D(64, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          500000    
                                                                 
 conv1d (Conv1D)             (None, 96, 64)            32064     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                        

In [9]:
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/10
513/513 [==============================] - 19s 34ms/step - loss: 0.5992 - accuracy: 0.7840 - val_loss: 0.4520 - val_accuracy: 0.8348
Epoch 2/10
513/513 [==============================] - 19s 36ms/step - loss: 0.3868 - accuracy: 0.8554 - val_loss: 0.4049 - val_accuracy: 0.8436
Epoch 3/10
513/513 [==============================] - 18s 34ms/step - loss: 0.2815 - accuracy: 0.8903 - val_loss: 0.4093 - val_accuracy: 0.8400
Epoch 4/10
513/513 [==============================] - 20s 39ms/step - loss: 0.1962 - accuracy: 0.9249 - val_loss: 0.5037 - val_accuracy: 0.8314
Epoch 5/10
513/513 [==============================] - 20s 39ms/step - loss: 0.1344 - accuracy: 0.9513 - val_loss: 0.5872 - val_accuracy: 0.8258
Epoch 6/10
513/513 [==============================] - 19s 37ms/step - loss: 0.0940 - accuracy: 0.9669 - val_loss: 0.7481 - val_accuracy: 0.8165
Epoch 7/10
513/513 [==============================] - 17s 34ms/step - loss: 0.0705 - accuracy: 0.9749 - val_loss: 0.8342 - val_accuracy:

In [10]:
y_pred = np.argmax(model.predict(x_test), axis=-1)
print("Accuracy:", accuracy_score(np.argmax(y_test, axis=-1), y_pred))

129/129 [==============================] - 1s 8ms/step
Accuracy: 0.8280068309343742


In [11]:
model.save('sentiment_analysis_model.h5')
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
# Load the saved model and tokenizer
import keras

model = keras.models.load_model('sentiment_analysis_model.h5')
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [13]:
def predict_sentiment(text):
    # Tokenize and pad the input text
    text_sequence = tokenizer.texts_to_sequences([text])
    text_sequence = pad_sequences(text_sequence, maxlen=100)

    # Make a prediction using the trained model
    predicted_rating = model.predict(text_sequence)[0]
    if np.argmax(predicted_rating) == 0:
        return 'Negative'
    elif np.argmax(predicted_rating) == 1:
        return 'Neutral'
    else:
        return 'Positive'

In [16]:
text_input = "I absolutely hated my stay at that hotel. The staff was worthless and the room was dirty"
predicted_sentiment = predict_sentiment(text_input)
print(predicted_sentiment)

1/1 [==============================] - 0s 25ms/step
Negative


In [17]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.0 MB/s eta 0:00:00


In [24]:
import pandas as pd

#create input and output directories




# get 1600 tweets
df = df[['Review','sentiment']].sample(n=1600, random_state=0)
df.to_csv("selected.csv", index=False)

In [22]:
df

,Review,sentiment
0,"great time, just returned breezes june 29th, r...",positive
1,amazing beautiful handsome hotel mr. o took be...,positive
2,service reading reviews finally feel need post...,positive
3,dingy rude aggressive receptionists hotel ok c...,negative
4,clean comfortable great price husband research...,positive
...,...,...
20486,best deal hong kong 89 paid ca n't imagine bet...,positive
20487,excellent hotel location excellent hotel locat...,positive
20488,great hotel not far christmas market came fran...,positive
20489,"benjamin great choice, stayed benjamin april 2...",positive


In [31]:
import re
from autocorrect import Speller
spell = Speller(lang='en')
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemm = WordNetLemmatizer()

#Fixing Word Lengthening
def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

def text_preprocess(doc):
    #Lowercasing all the letters
    temp = doc.lower()
    #Removing hashtags and mentions
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    #Removing links
    temp = re.sub(r"http\S+", "", temp)
    temp = re.sub(r"www.\S+", "", temp)
    #removing numbers
    temp = re.sub("[0-9]","", temp)
    #Removing '
    temp = re.sub("'"," ",temp)

    #Tokenization
    temp = word_tokenize(temp)
    #Fixing Word Lengthening
    temp = [reduce_lengthening(w) for w in temp]
    #spell corrector
    temp = [spell(w) for w in temp]
    #stem
    temp = [lemm.lemmatize(w) for w in temp]
    #Removing short words
    temp = [w for w in temp if len(w)>2]
    temp = " ".join(w for w in temp)

    return temp

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
